In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv')

C:\Users\Selim\AppData\Local\Temp\ipykernel_16324\2949129822.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


In [3]:
train = train.drop(columns = ['MachineIdentifier', 'AvSigVersion', 'IsBeta', 'RtpStateBitfield', 'DefaultBrowsersIdentifier', 'AVProductsInstalled', 
      'HasTpm', 'LocaleEnglishNameIdentifier', 'PuaMode', 'SMode', 'SmartScreen', 'UacLuaenable', 'Census_DeviceFamily', 'Census_ProcessorClass',
      'Census_InternalBatteryType', 'Census_IsPortableOperatingSystem', 'Census_IsFlightingInternal', 'Census_ThresholdOptIn',
       'OrganizationIdentifier', 'Census_IsWIMBootEnabled'])
train.shape
#Preprocessing to remove unusable variables
#cols to drop because they are either id's that are too spread out, have only 1 value or contain 30%+ missing data 
train = train.sample(1784296)
train = train.dropna()

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [5]:
#Find the variables that have too many classes to OHE

for i in train.columns:
    if pd.get_dummies(train[[i]]).shape[1] > 30:
        print(i)
#We will use target encoding to replace these, or preprocess them in a different way
target = train['HasDetections'].mean()
train['EngineVersion'] = train['EngineVersion'].str.replace('.', '').astype(int)
train['AppVersion'] = train['AppVersion'].shape[0] * [target]
train['OsBuildLab'] = train['OsBuildLab'].shape[0] * [target]
train['Census_ChassisTypeName'] = train['Census_ChassisTypeName'].shape[0] * [target]
train['Census_OSVersion'] = train['Census_OSVersion'].shape[0] * [target]

EngineVersion
AppVersion
OsBuildLab
Census_ChassisTypeName
Census_OSVersion


C:\Users\Selim\AppData\Local\Temp\ipykernel_16324\1363444013.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['EngineVersion'] = train['EngineVersion'].str.replace('.', '').astype(int)


In [6]:
X_dev, X_test, y_dev, y_test = train_test_split(train.iloc[:, 0:-1], train['HasDetections'],test_size = 0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev,test_size = 0.25, random_state = 42)


In [7]:
X_train = pd.get_dummies(X_train)
X_val = pd.get_dummies(X_val)
X_test = pd.get_dummies(X_test)


In [8]:
a = np.intersect1d(X_train.columns, X_val.columns)
X_train = X_train[a]
X_val = X_val[a]
X_train.shape, X_val.shape, X_test.shape

((919411, 179), (306471, 179), (306471, 183))

In [9]:
a = np.intersect1d(X_train.columns, X_test.columns)
X_test = X_test[a]
X_train = X_train[a]
X_val = X_val[a]
X_train.shape, X_val.shape, X_test.shape


((919411, 177), (306471, 177), (306471, 177))

In [10]:
clf = RandomForestClassifier(random_state = 42)
clf = clf.fit(X_train, y_train)
results_train= (clf.predict(X_train) == y_train)
print('Accuracy on train set' , results_train.sum()/results_train.shape[0])
results_test = (clf.predict(X_val) == y_val)
print('Accuracy on val set' , results_test.sum()/results_test.shape[0])

Accuracy on train set 0.9996965448531723
Accuracy on val set 0.6169099196987643


Obviously we are overfitting on the train set here, so lets perform some CV with random search

In [11]:
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import RandomizedSearchCV

In [12]:
clf = RandomForestClassifier(random_state = 42)
parameters = {'criterion': ['gini', 'entropy'],'max_depth':sp_randInt(5,25).rvs(5), 'min_impurity_decrease':  sp_randFloat(0,0.2).rvs(5)}
rand = RandomizedSearchCV(estimator=clf, param_distributions = parameters, 
                               cv = 5, n_iter = 5, n_jobs=-1, scoring = 'accuracy')
search = rand.fit(X_train, y_train)
search.best_params_

{'min_impurity_decrease': 0.07993430241516873,
 'max_depth': 5,
 'criterion': 'gini'}

In [13]:
clf = RandomForestClassifier(random_state = 42 , max_depth = search.best_params_['max_depth'], min_impurity_decrease = search.best_params_['min_impurity_decrease'],
                            criterion = search.best_params_['criterion'])
clf = clf.fit(X_train, y_train)
results_train= (clf.predict(X_train) == y_train)
print('Accuracy on train set' , results_train.sum()/results_train.shape[0])
results_test = (clf.predict(X_val) == y_val)
print('Accuracy on val set' , results_test.sum()/results_test.shape[0])
results_final= (clf.predict(X_test) == y_test)
print('Accuracy on train set' , results_final.sum()/results_final.shape[0])

Accuracy on train set 0.5008804549869428
Accuracy on val set 0.5010816684123457
Accuracy on train set 0.5006574847212297


In [14]:
results_final= (clf.predict(X_test) == y_test)
print('Accuracy on train set' , results_final.sum()/results_final.shape[0])

Accuracy on train set 0.5006574847212297


In [15]:
clf = RandomForestClassifier(random_state = 42)
parameters = {'criterion': ['gini', 'entropy'],'max_depth':sp_randInt(5,25).rvs(5), 'min_samples_split':  sp_randInt(2,100).rvs(5)}
rand = RandomizedSearchCV(estimator=clf, param_distributions = parameters, 
                               cv = 5, n_iter = 5, n_jobs=-1, scoring = 'accuracy')
search = rand.fit(X_train, y_train)
search.best_params_

{'min_samples_split': 67, 'max_depth': 17, 'criterion': 'entropy'}

In [16]:
clf = RandomForestClassifier(random_state = 42 , max_depth = search.best_params_['max_depth'], min_samples_split = search.best_params_['min_samples_split'],
                            criterion = search.best_params_['criterion'], )
clf = clf.fit(X_train, y_train)
results_train= (clf.predict(X_train) == y_train)
print('Accuracy on train set' , results_train.sum()/results_train.shape[0])
results_test = (clf.predict(X_val) == y_val)
print('Accuracy on val set' , results_test.sum()/results_test.shape[0])
results_final= (clf.predict(X_test) == y_test)
print('Accuracy on train set' , results_final.sum()/results_final.shape[0])

Accuracy on train set 0.6411343784227076
Accuracy on val set 0.6194680736513406
Accuracy on train set 0.6183227776853275
